In [33]:
#Imports and API Key
#building in offsets

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [34]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [35]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [36]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [37]:
#converting GDP quarterly data into monthly - need to convert it so it fills in the following 3 months

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [38]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [39]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [40]:
pd.set_option('display.max_columns', 500)

In [41]:
offset_dict = {'PMI':1, 'UNR':1, 'YUNR':2,'HOME_SALES':2, 'PART_TIME':1, 'CPPR_PRICE':2, 'HOUS_PERMS':1, 'HOUS_STARTS':1, 'CAP_UTIL':1, 'PERS_SAVINGS':2, 'EXPORTS':2, 'IMPORTS':2, 'TRADE_BALANCE':2, 'US_M2':2, 'US_NHOME_SALES':2,'PPI':2,'CPI':2} #positives shift down, negatives shift up. Goal of this dict is to align data to when it gets released

In [42]:
dataset.tail(100)

,PMI,UNR,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,EXPORTS,IMPORTS,TRADE_BALANCE,INT_RATE,US_M2,US_NHOME_SALES,PPI,CPI,3YRT,Recession,10 Year Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread
2011-11,51.9,8.6,17.1,15796.460,63.7,4390.0,2476.0,452.1,706.0,711.0,76.6057,7.0,179589.0,226633.0,-47044.0,0.08,9596.3,328.0,201.4,103.748816,7.87,0.0,2.01,0.01,2.00
2011-12,52.9,8.5,16.7,15796.460,69.9,4350.0,2397.0,449.5,697.0,694.0,76.8861,7.8,180513.0,229973.0,-49460.0,0.07,9635.9,341.0,199.8,103.492918,8.13,0.0,1.98,0.01,1.97
2012-01,53.3,8.3,16.1,16019.758,75.0,4480.0,2563.0,452.7,712.0,723.0,77.2097,8.0,180073.0,231056.0,-50983.0,0.08,9714.8,335.0,200.7,103.948307,7.16,0.0,1.97,0.03,1.94
2012-02,53.4,8.3,16.5,16019.758,75.3,4580.0,2370.0,478.1,738.0,704.0,77.2402,8.0,182640.0,226210.0,-43570.0,0.10,9758.1,366.0,201.6,104.405988,7.69,0.0,1.97,0.09,1.88
2012-03,53.6,8.2,16.3,16019.758,76.2,4520.0,2344.0,479.4,806.0,695.0,76.7353,8.5,186316.0,236533.0,-50218.0,0.13,9801.4,354.0,204.2,105.198906,11.17,0.0,2.17,0.08,2.09
2012-04,54.0,8.2,16.5,16152.257,76.4,4590.0,2406.0,467.8,732.0,753.0,77.1696,8.7,185030.0,232284.0,-47254.0,0.14,9849.0,354.0,203.7,105.516715,9.07,0.0,2.05,0.08,1.97
2012-05,53.1,8.2,16.1,16152.257,79.3,4600.0,2540.0,457.0,796.0,708.0,77.1784,8.8,185070.0,231041.0,-45970.0,0.16,9883.1,370.0,201.9,105.392893,8.54,0.0,1.80,0.09,1.71
2012-06,51.1,8.2,16.3,16152.257,73.2,4470.0,2492.0,435.9,794.0,757.0,77.0278,9.1,185286.0,228746.0,-43460.0,0.16,9951.0,360.0,199.8,105.238345,8.17,0.0,1.62,0.09,1.53
2012-07,50.0,8.2,16.3,16257.151,72.3,4530.0,2551.0,438.7,849.0,740.0,77.0989,8.2,184336.0,227706.0,-43370.0,0.16,10029.9,369.0,200.1,105.066829,6.95,0.0,1.53,0.10,1.43
2012-08,50.3,8.1,16.7,16257.151,74.3,4760.0,2447.0,434.6,840.0,754.0,76.6028,8.0,183265.0,227372.0,-44107.0,0.13,10098.3,375.0,202.7,105.651543,8.46,0.0,1.68,0.10,1.58


In [43]:
for entry in offset_dict:
    #print(entry)
    #print(offset_dict[entry])
    dataset[str(entry)] = dataset[str(entry)].shift(offset_dict[entry])

In [44]:
dataset.tail(100)

,PMI,UNR,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,EXPORTS,IMPORTS,TRADE_BALANCE,INT_RATE,US_M2,US_NHOME_SALES,PPI,CPI,3YRT,Recession,10 Year Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread
2011-11,51.5,8.8,17.3,15796.460,63.7,4330.0,2500.0,488.6,671.0,610.0,76.8067,6.8,182398.0,225703.0,-43305.0,0.08,9522.2,304.0,203.7,104.051033,7.87,0.0,2.01,0.01,2.00
2011-12,51.9,8.6,16.7,15796.460,69.9,4340.0,2476.0,442.8,706.0,711.0,76.6057,6.8,181683.0,226645.0,-44962.0,0.07,9553.6,316.0,201.1,103.836409,8.13,0.0,1.98,0.01,1.97
2012-01,52.9,8.5,17.1,16019.758,75.0,4390.0,2397.0,452.1,697.0,694.0,76.8861,7.0,179589.0,226633.0,-47044.0,0.08,9596.3,328.0,201.4,103.748816,7.16,0.0,1.97,0.03,1.94
2012-02,53.3,8.3,16.7,16019.758,75.3,4350.0,2563.0,449.5,712.0,723.0,77.2097,7.8,180513.0,229973.0,-49460.0,0.10,9635.9,341.0,199.8,103.492918,7.69,0.0,1.97,0.09,1.88
2012-03,53.4,8.3,16.1,16019.758,76.2,4480.0,2370.0,452.7,738.0,704.0,77.2402,8.0,180073.0,231056.0,-50983.0,0.13,9714.8,335.0,200.7,103.948307,11.17,0.0,2.17,0.08,2.09
2012-04,53.6,8.2,16.5,16152.257,76.4,4580.0,2344.0,478.1,806.0,695.0,76.7353,8.0,182640.0,226210.0,-43570.0,0.14,9758.1,366.0,201.6,104.405988,9.07,0.0,2.05,0.08,1.97
2012-05,54.0,8.2,16.3,16152.257,79.3,4520.0,2406.0,479.4,732.0,753.0,77.1696,8.5,186316.0,236533.0,-50218.0,0.16,9801.4,354.0,204.2,105.198906,8.54,0.0,1.80,0.09,1.71
2012-06,53.1,8.2,16.5,16152.257,73.2,4590.0,2540.0,467.8,796.0,708.0,77.1784,8.7,185030.0,232284.0,-47254.0,0.16,9849.0,354.0,203.7,105.516715,8.17,0.0,1.62,0.09,1.53
2012-07,51.1,8.2,16.1,16257.151,72.3,4600.0,2492.0,457.0,794.0,757.0,77.0278,8.8,185070.0,231041.0,-45970.0,0.16,9883.1,370.0,201.9,105.392893,6.95,0.0,1.53,0.10,1.43
2012-08,50.0,8.2,16.3,16257.151,74.3,4470.0,2551.0,435.9,849.0,740.0,77.0989,9.1,185286.0,228746.0,-43460.0,0.13,9951.0,360.0,199.8,105.238345,8.46,0.0,1.68,0.10,1.58


In [23]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [24]:
y = dataset['Recession'] #splitting off Y

In [25]:
y_shift = y.shift(-3) #shifting y to forecast 3 months out

In [26]:
y_shift = y_shift.fillna(0)

In [27]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [28]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [29]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [30]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [31]:
X = dataset

In [ ]:
### Data Prep Finished Here ###

In [32]:
dataset.tail()

,PMI,UNR,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,EXPORTS,IMPORTS,TRADE_BALANCE,INT_RATE,US_M2,US_NHOME_SALES,PPI,CPI,3YRT,10 Year Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread,HOME_SALES_PXY,CPPR_PRICE_PXY,HOUS_PERMS_PXY,HOUS_STARTS_PXY,CAP_UTIL_PXY,PERS_SAVINGS_PXY,US_M2_PXY,US_NHOME_SALES_PXY,3YRT_PXY,PMI_1m_shift,UNR_1m_shift,YUNR_1m_shift,CONS_SENT_1m_shift,HOME_SALES_1m_shift,PART_TIME_1m_shift,CPPR_PRICE_1m_shift,HOUS_PERMS_1m_shift,HOUS_STARTS_1m_shift,CAP_UTIL_1m_shift,PERS_SAVINGS_1m_shift,EXPORTS_1m_shift,IMPORTS_1m_shift,TRADE_BALANCE_1m_shift,INT_RATE_1m_shift,US_M2_1m_shift,US_NHOME_SALES_1m_shift,3YRT_1m_shift,10 Year Treasury Yield_1m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift,Spread_1m_shift,HOME_SALES_PXY_1m_shift,CPPR_PRICE_PXY_1m_shift,HOUS_PERMS_PXY_1m_shift,PMI_3m_shift,UNR_3m_shift,YUNR_3m_shift,CONS_SENT_3m_shift,HOME_SALES_3m_shift,PART_TIME_3m_shift,CPPR_PRICE_3m_shift,HOUS_PERMS_3m_shift,HOUS_STARTS_3m_shift,CAP_UTIL_3m_shift,PERS_SAVINGS_3m_shift,EXPORTS_3m_shift,IMPORTS_3m_shift,TRADE_BALANCE_3m_shift,INT_RATE_3m_shift,US_M2_3m_shift,US_NHOME_SALES_3m_shift,3YRT_3m_shift,10 Year Treasury Yield_3m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_3m_shift,Spread_3m_shift,HOME_SALES_PXY_3m_shift,CPPR_PRICE_PXY_3m_shift,HOUS_PERMS_PXY_3m_shift,PMI_12m_shift,UNR_12m_shift,YUNR_12m_shift,CONS_SENT_12m_shift,HOME_SALES_12m_shift,PART_TIME_12m_shift,CPPR_PRICE_12m_shift,HOUS_PERMS_12m_shift,HOUS_STARTS_12m_shift,CAP_UTIL_12m_shift,PERS_SAVINGS_12m_shift,EXPORTS_12m_shift,IMPORTS_12m_shift,TRADE_BALANCE_12m_shift,INT_RATE_12m_shift,US_M2_12m_shift,US_NHOME_SALES_12m_shift,3YRT_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift,HOME_SALES_PXY_12m_shift,CPPR_PRICE_PXY_12m_shift,HOUS_PERMS_PXY_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
2014-11,58.2,6.2,13.3,88.8,4900.0,2609.0,407.7,1099.0,1077.0,78.7287,7.4,197418.0,239485.0,-42068.0,0.09,11481.4,466.0,206.4,109.160741,17.33,2.33,0.02,2.31,0,0,0,0,0,0,0,0,0,2.9,0.0,0.3,1.9,110.0,66.0,5.9,27.0,57.0,-0.0893,0.2,-2502.0,1420.0,-3923.0,0.00,37.8,11.0,-1.92,0.03,0.00,0.03,0.0,0.0,0.0,2.4,-0.1,-1.2,6.3,120.0,210.0,-14.6,27.0,-5.0,-0.1587,0.0,-1735.0,567.0,-2303.0,0.00,150.7,50.0,-2.15,-0.09,-0.01,-0.08,0.0,0.0,0.0,2.9,-1.0,-2.9,13.7,-240.0,-43.0,-17.0,58.0,152.0,1.4413,0.6,6627.0,7377.0,-751.0,0.01,672.0,63.0,6.31,-0.39,-0.05,-0.34,0.0,0.0,0.0,0.075257,-0.289855,-0.130904,-0.912146,1.657919,1.226091
2014-12,55.5,5.9,13.1,93.6,4940.0,2545.0,405.1,1063.0,1001.0,79.2428,7.2,200144.0,240772.0,-40629.0,0.12,11521.7,472.0,203.4,108.886499,23.41,2.21,0.03,2.18,0,0,0,0,0,0,0,0,0,-2.7,-0.3,-0.2,4.8,40.0,-64.0,-2.6,-36.0,-76.0,0.5141,-0.2,2726.0,1287.0,1439.0,0.03,40.3,6.0,6.08,-0.12,0.01,-0.13,0.0,0.0,0.0,0.6,-0.2,-1.5,9.0,190.0,-112.0,-7.1,21.0,18.0,0.5520,-0.3,654.0,1535.0,-883.0,0.03,115.9,70.0,1.34,-0.32,0.01,-0.33,0.0,0.0,0.0,0.1,-1.3,-3.0,11.1,-170.0,-17.0,-14.6,32.0,-99.0,1.7934,0.9,5802.0,8629.0,-2828.0,0.03,601.3,28.0,9.02,-0.69,-0.04,-0.65,0.0,0.0,0.0,-0.251228,-1.453488,-0.342917,-2.211538,1.664340,0.444444
2015-01,56.8,5.7,13.5,98.1,4980.0,2425.0,415.9,1077.0,1068.0,79.0322,7.3,198017.0,237607.0,-39591.0,0.11,11564.7,444.0,200.9,108.298575,17.94,1.88,0.03,1.85,0,0,0,0,0,0,0,0,0,1.3,-0.2,0.4,4.5,40.0,-120.0,10.8,14.0,67.0,-0.2106,0.1,-2127.0,-3165.0,1038.0,-0.01,43.0,-28.0,-5.47,-0.33,0.00,-0.33,0.0,0.0,0.0,1.5,-0.5,0.5,11.2,190.0,-118.0,14.1,5.0,48.0,0.2142,0.1,-1903.0,-458.0,-1446.0,0.02,121.1,-11.0,-1.31,-0.42,0.01,-0.43,0.0,0.0,0.0,1.4,-1.5,-2.0,16.9,-290.0,-204.0,5.5,68.0,66.0,1.4130,1.1,2187.0,6926.0,-4740.0,0.04,634.9,-2.0,1.57,-0.98,-0.01,-0.97,0.0,0.0,0.0,-0.539942,-1.229105,-0.715151,-2.946860,1.322355,-0.149105
2015-02,57.0,5.8,13.2,95.4,4970.0,2417.0,413.6,1060.0,1094.0,78.5510,7.6,197177.0,239813.0,-42636.0,0.11,11630.5,488.0,197.0,107.684512,18